# 🧪 Trello MCP Cloud Run Test Suite

This notebook provides interactive testing for the **Trello MCP Server running on Google Cloud Run**.

## Features Tested:
- ✅ Health check endpoint
- ✅ Interactive login feature (`/auth/login`)
- ✅ Tools endpoint with optional credentials
- ✅ MCP protocol compliance
- ✅ All 11 Trello tools

**Service URL:** https://trello-mcp-116435607783.us-central1.run.app

## Section 1: Import Required Libraries and Dependencies

In [19]:
import asyncio
import json
import unittest
from typing import Dict, Any, Optional
import sys
from pathlib import Path

# For HTTP requests
try:
    import requests
except ImportError:
    print("Installing requests...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "requests", "-q"])
    import requests

print("✅ All required libraries imported successfully!")
print(f"Requests version: {requests.__version__}")

✅ All required libraries imported successfully!
Requests version: 2.32.5


## Section 2: Configure Cloud Run Service URL and Session

In [20]:
# Cloud Run Service Configuration
SERVICE_URL = "https://trello-mcp-116435607783.us-central1.run.app"

# Create a session for HTTP requests
session = requests.Session()
session.timeout = 10

print(f"📍 Service URL: {SERVICE_URL}")
print(f"⏱️ Request timeout: {session.timeout} seconds")
print("\n✅ Configuration ready!")

📍 Service URL: https://trello-mcp-116435607783.us-central1.run.app
⏱️ Request timeout: 10 seconds

✅ Configuration ready!


## Section 3: Connect to Trello MCP Endpoint and Verify Connectivity

In [21]:
# Test 1: Health Check Endpoint
print("="*80)
print("🏥 TEST 1: Health Check Endpoint")
print("="*80)

try:
    response = session.get(f"{SERVICE_URL}/health", timeout=10)
    response.raise_for_status()
    
    health_data = response.json()
    print("✅ Connected to Trello MCP service!")
    print("\n📋 Health Status Response:")
    print(json.dumps(health_data, indent=2))
    
    # Validate health
    if health_data.get("status") == "healthy":
        print("\n✅ Service is HEALTHY")
    else:
        print("\n❌ Service health status:", health_data.get("status"))
        
    tools_count = health_data.get("tools_count", 0)
    print(f"✅ Available tools: {tools_count}")
    
except Exception as e:
    print(f"❌ Failed to connect: {e}")
    print("⚠️  Make sure the Cloud Run service is running and accessible")

🏥 TEST 1: Health Check Endpoint
✅ Connected to Trello MCP service!

📋 Health Status Response:
{
  "status": "healthy",
  "service": "trello-mcp",
  "version": "1.0.0",
  "timestamp": "2025-10-27T11:48:58.636073+00:00",
  "mcp_server": "initialized",
  "tools_count": 11
}

✅ Service is HEALTHY
✅ Available tools: 11
✅ Connected to Trello MCP service!

📋 Health Status Response:
{
  "status": "healthy",
  "service": "trello-mcp",
  "version": "1.0.0",
  "timestamp": "2025-10-27T11:48:58.636073+00:00",
  "mcp_server": "initialized",
  "tools_count": 11
}

✅ Service is HEALTHY
✅ Available tools: 11


## Section 4: Retrieve and Display Trello MCP Data

In [22]:
# Test 2: Root Endpoint
print("\n" + "="*80)
print("🌐 TEST 2: Root Endpoint")
print("="*80)

try:
    response = session.get(f"{SERVICE_URL}/", timeout=10)
    response.raise_for_status()
    
    root_data = response.json()
    print("✅ Root endpoint responding")
    print("\n📋 Root Response:")
    print(json.dumps(root_data, indent=2))
    
except Exception as e:
    print(f"❌ Root endpoint failed: {e}")

# Test 3: Interactive Login Feature
print("\n" + "="*80)
print("🔐 TEST 3: Interactive Login Feature (/auth/login)")
print("="*80)

try:
    response = session.get(f"{SERVICE_URL}/auth/login", timeout=10)
    response.raise_for_status()
    
    login_data = response.json()
    print("✅ Interactive login endpoint responding")
    print("\n📋 Login Feature Response:")
    print(json.dumps(login_data, indent=2))
    
    features = login_data.get("features", {})
    print(f"\n✅ Browser auto-open: {features.get('automatic_browser_open')}")
    print(f"✅ Session caching: {features.get('session_caching')}")
    print(f"✅ Cache duration: {features.get('cache_duration_minutes')} minutes")
    
except Exception as e:
    print(f"❌ Login endpoint failed: {e}")


🌐 TEST 2: Root Endpoint
✅ Root endpoint responding

📋 Root Response:
{
  "name": "Trello MCP Server",
  "description": "Model Context Protocol server for Trello API integration",
  "version": "1.0.0",
  "endpoints": {
    "health": "/health",
    "login": "/auth/login (GET)",
    "mcp": "/mcp (POST)",
    "tools": "/tools (GET)"
  },
  "features": {
    "interactive_login": true,
    "session_credentials_caching": true,
    "no_persistent_storage": true
  },
  "documentation": "https://github.com/Skai-IT/trello-mcp-server"
}

🔐 TEST 3: Interactive Login Feature (/auth/login)
✅ Interactive login endpoint responding

📋 Login Feature Response:
{
  "message": "Interactive login will be triggered on first tool call",
  "login_url": "https://trello.com/app-key",
  "instructions": {
    "step_1": "Visit https://trello.com/app-key",
    "step_2": "Copy your API Key",
    "step_3": "Click 'Token' link to generate/view your token",
    "step_4": "When prompted by the MCP server, paste both valu

In [23]:
# Test 4: Tools Endpoint
print("\n" + "="*80)
print("🛠️ TEST 4: Available Tools")
print("="*80)

try:
    response = session.get(f"{SERVICE_URL}/tools", timeout=10)
    response.raise_for_status()
    
    tools_data = response.json()
    tools = tools_data.get("tools", [])
    
    print(f"✅ Found {len(tools)} available tools")
    
    # List all tools
    print("\n📋 Available Tools:")
    for i, tool in enumerate(tools, 1):
        name = tool.get("name")
        description = tool.get("description", "")
        schema = tool.get("inputSchema", {})
        required = schema.get("required", [])
        
        # Check if credentials are required
        creds_required = any(k in required for k in ["api_key", "token"])
        creds_status = "❌ Required" if creds_required else "✅ Optional"
        
        print(f"\n{i}. {name}")
        print(f"   Description: {description[:60]}...")
        print(f"   Credentials: {creds_status}")
    
except Exception as e:
    print(f"❌ Tools endpoint failed: {e}")


🛠️ TEST 4: Available Tools
✅ Found 11 available tools

📋 Available Tools:

1. list_boards
   Description: List all boards for the authenticated user (credentials opti...
   Credentials: ✅ Optional

2. get_board
   Description: Get detailed information about a specific board...
   Credentials: ✅ Optional

3. create_board
   Description: Create a new board...
   Credentials: ✅ Optional

4. update_board
   Description: Update an existing board...
   Credentials: ✅ Optional

5. get_lists
   Description: Get all lists on a board...
   Credentials: ✅ Optional

6. create_list
   Description: Create a new list on a board...
   Credentials: ✅ Optional

7. get_cards
   Description: Get cards from a board or list...
   Credentials: ✅ Optional

8. create_card
   Description: Create a new card...
   Credentials: ✅ Optional

9. update_card
   Description: Update an existing card...
   Credentials: ✅ Optional

10. add_member_to_card
   Description: Add a member to a card...
   Credentials: ✅ Optiona

## Section 5: Test MCP Protocol Endpoints

In [24]:
# Test 5: MCP Initialize Request
print("\n" + "="*80)
print("🚀 TEST 5: MCP Initialize Request")
print("="*80)

mcp_request = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "protocolVersion": "2024-11-05",
        "clientInfo": {
            "name": "vscode-test",
            "version": "1.0.0"
        }
    }
}

try:
    response = session.post(
        f"{SERVICE_URL}/mcp",
        json=mcp_request,
        timeout=10,
        headers={"Content-Type": "application/json"}
    )
    response.raise_for_status()
    
    mcp_response = response.json()
    print("✅ MCP Initialize request successful")
    print("\n📋 MCP Initialize Response:")
    print(json.dumps(mcp_response, indent=2))
    
except Exception as e:
    print(f"❌ MCP Initialize failed: {e}")

# Test 6: MCP Tools List Request
print("\n" + "="*80)
print("🚀 TEST 6: MCP Tools/List Request")
print("="*80)

mcp_tools_request = {
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/list"
}

try:
    response = session.post(
        f"{SERVICE_URL}/mcp",
        json=mcp_tools_request,
        timeout=10,
        headers={"Content-Type": "application/json"}
    )
    response.raise_for_status()
    
    mcp_tools_response = response.json()
    result = mcp_tools_response.get("result", {})
    tools_list = result.get("tools", [])
    
    print("✅ MCP Tools/List request successful")
    print(f"\n✅ MCP Server has {len(tools_list)} tools available")
    print("\n📋 First 3 tools from MCP response:")
    for tool in tools_list[:3]:
        print(f"  - {tool.get('name')}: {tool.get('description', 'N/A')[:50]}...")
    
except Exception as e:
    print(f"❌ MCP Tools/List failed: {e}")


🚀 TEST 5: MCP Initialize Request
✅ MCP Initialize request successful

📋 MCP Initialize Response:
{
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "protocolVersion": "2024-11-05",
    "capabilities": {
      "tools": {
        "listChanged": false
      },
      "resources": {},
      "prompts": {}
    },
    "serverInfo": {
      "name": "trello-mcp",
      "version": "1.0.0"
    }
  }
}

🚀 TEST 6: MCP Tools/List Request
✅ MCP Tools/List request successful

✅ MCP Server has 11 tools available

📋 First 3 tools from MCP response:
  - list_boards: List all boards for the authenticated user (creden...
  - get_board: Get detailed information about a specific board...
  - create_board: Create a new board...
✅ MCP Tools/List request successful

✅ MCP Server has 11 tools available

📋 First 3 tools from MCP response:
  - list_boards: List all boards for the authenticated user (creden...
  - get_board: Get detailed information about a specific board...
  - create_board: Create a new board...


## Section 6: Create Unit Tests for API Responses

In [25]:
class TestTrelloMCPCloudRun(unittest.TestCase):
    """Unit tests for Trello MCP Cloud Run service"""
    
    @classmethod
    def setUpClass(cls):
        """Set up test fixtures"""
        cls.service_url = SERVICE_URL
        cls.session = requests.Session()
        cls.session.timeout = 10
    
    def test_01_health_endpoint(self):
        """Test health endpoint returns 200 and correct structure"""
        response = self.session.get(f"{self.service_url}/health", timeout=10)
        self.assertEqual(response.status_code, 200, "Health endpoint should return 200")
        
        data = response.json()
        self.assertEqual(data.get("status"), "healthy", "Status should be 'healthy'")
        self.assertEqual(data.get("tools_count"), 11, "Should have 11 tools")
        self.assertIn("mcp_server", data, "Response should contain mcp_server key")
    
    def test_02_root_endpoint(self):
        """Test root endpoint and feature flags"""
        response = self.session.get(f"{self.service_url}/", timeout=10)
        self.assertEqual(response.status_code, 200, "Root endpoint should return 200")
        
        data = response.json()
        self.assertIn("endpoints", data, "Should have endpoints")
        self.assertIn("features", data, "Should have features")
        
        features = data.get("features", {})
        self.assertTrue(features.get("interactive_login"), "Interactive login should be enabled")
        self.assertTrue(features.get("session_credentials_caching"), "Session caching should be enabled")
    
    def test_03_auth_login_endpoint(self):
        """Test interactive login endpoint"""
        response = self.session.get(f"{self.service_url}/auth/login", timeout=10)
        self.assertEqual(response.status_code, 200, "/auth/login should return 200")
        
        data = response.json()
        self.assertIn("login_url", data, "Should have login_url")
        self.assertIn("instructions", data, "Should have instructions")
        self.assertIn("features", data, "Should have features")
        
        features = data.get("features", {})
        self.assertTrue(features.get("automatic_browser_open"), "Browser auto-open should be enabled")
        self.assertTrue(features.get("session_caching"), "Session caching should be enabled")
        self.assertEqual(features.get("cache_duration_minutes"), 480, "Cache should be 8 hours (480 minutes)")
    
    def test_04_tools_endpoint(self):
        """Test tools endpoint has all 11 tools"""
        response = self.session.get(f"{self.service_url}/tools", timeout=10)
        self.assertEqual(response.status_code, 200, "/tools should return 200")
        
        data = response.json()
        tools = data.get("tools", [])
        self.assertEqual(len(tools), 11, "Should have exactly 11 tools")
        
        # Check tool names
        expected_tools = [
            "list_boards", "get_board", "create_board", "update_board",
            "get_lists", "create_list",
            "get_cards", "create_card", "update_card",
            "add_member_to_card", "search_cards"
        ]
        tool_names = [t.get("name") for t in tools]
        for tool_name in expected_tools:
            self.assertIn(tool_name, tool_names, f"Should have {tool_name} tool")
    
    def test_05_tools_optional_credentials(self):
        """Test that credentials are optional for all tools"""
        response = self.session.get(f"{self.service_url}/tools", timeout=10)
        data = response.json()
        tools = data.get("tools", [])
        
        for tool in tools:
            name = tool.get("name")
            schema = tool.get("inputSchema", {})
            required = schema.get("required", [])
            
            # api_key and token should NOT be in required list
            has_api_key = "api_key" in required
            has_token = "token" in required
            
            self.assertFalse(has_api_key, f"{name} should not require api_key")
            self.assertFalse(has_token, f"{name} should not require token")
    
    def test_06_mcp_initialize(self):
        """Test MCP initialize request"""
        mcp_request = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "initialize",
            "params": {
                "protocolVersion": "2024-11-05",
                "clientInfo": {"name": "test", "version": "1.0"}
            }
        }
        
        response = self.session.post(
            f"{self.service_url}/mcp",
            json=mcp_request,
            timeout=10,
            headers={"Content-Type": "application/json"}
        )
        self.assertEqual(response.status_code, 200, "MCP endpoint should return 200")
        
        data = response.json()
        self.assertEqual(data.get("jsonrpc"), "2.0", "Should be JSON-RPC 2.0")
        self.assertIn("result", data, "Should have result")
        
        result = data.get("result", {})
        self.assertIn("capabilities", result, "Result should have capabilities")
    
    def test_07_mcp_tools_list(self):
        """Test MCP tools/list request"""
        mcp_request = {
            "jsonrpc": "2.0",
            "id": 2,
            "method": "tools/list"
        }
        
        response = self.session.post(
            f"{self.service_url}/mcp",
            json=mcp_request,
            timeout=10,
            headers={"Content-Type": "application/json"}
        )
        self.assertEqual(response.status_code, 200, "MCP endpoint should return 200")
        
        data = response.json()
        result = data.get("result", {})
        tools = result.get("tools", [])
        
        self.assertEqual(len(tools), 11, "MCP should report 11 tools")

print("✅ Unit test class defined successfully!")
print(f"📊 Test methods: {len([m for m in dir(TestTrelloMCPCloudRun) if m.startswith('test_')])}")

✅ Unit test class defined successfully!
📊 Test methods: 7


## Section 7: Save Tests to VSCode Project Directory

In [26]:
# Save unit tests to file
test_file_path = Path("/Users/shlomisha/Documents/vscodeprojects/Trello/tests/test_cloud_run_integration.py")
test_file_path.parent.mkdir(parents=True, exist_ok=True)

test_code = '''#!/usr/bin/env python3
"""
Unit tests for Trello MCP Cloud Run service

Run tests with:
    python -m pytest tests/test_cloud_run_integration.py -v
    
Or with unittest:
    python -m unittest tests.test_cloud_run_integration -v
"""

import unittest
import requests
from typing import Dict, Any

SERVICE_URL = "https://trello-mcp-116435607783.us-central1.run.app"


class TestTrelloMCPCloudRun(unittest.TestCase):
    """Unit tests for Trello MCP Cloud Run service"""
    
    @classmethod
    def setUpClass(cls):
        """Set up test fixtures"""
        cls.service_url = SERVICE_URL
        cls.session = requests.Session()
        cls.session.timeout = 10
    
    def test_01_health_endpoint(self):
        """Test health endpoint returns 200 and correct structure"""
        response = self.session.get(f"{self.service_url}/health", timeout=10)
        self.assertEqual(response.status_code, 200, "Health endpoint should return 200")
        
        data = response.json()
        self.assertEqual(data.get("status"), "healthy", "Status should be 'healthy'")
        self.assertEqual(data.get("tools_count"), 11, "Should have 11 tools")
        self.assertIn("mcp_server", data, "Response should contain mcp_server key")
    
    def test_02_root_endpoint(self):
        """Test root endpoint and feature flags"""
        response = self.session.get(f"{self.service_url}/", timeout=10)
        self.assertEqual(response.status_code, 200, "Root endpoint should return 200")
        
        data = response.json()
        self.assertIn("endpoints", data, "Should have endpoints")
        self.assertIn("features", data, "Should have features")
        
        features = data.get("features", {})
        self.assertTrue(features.get("interactive_login"), "Interactive login should be enabled")
        self.assertTrue(features.get("session_credentials_caching"), "Session caching should be enabled")
    
    def test_03_auth_login_endpoint(self):
        """Test interactive login endpoint"""
        response = self.session.get(f"{self.service_url}/auth/login", timeout=10)
        self.assertEqual(response.status_code, 200, "/auth/login should return 200")
        
        data = response.json()
        self.assertIn("login_url", data, "Should have login_url")
        self.assertIn("instructions", data, "Should have instructions")
        self.assertIn("features", data, "Should have features")
        
        features = data.get("features", {})
        self.assertTrue(features.get("automatic_browser_open"), "Browser auto-open should be enabled")
        self.assertTrue(features.get("session_caching"), "Session caching should be enabled")
        self.assertEqual(features.get("cache_duration_minutes"), 480, "Cache should be 8 hours (480 minutes)")
    
    def test_04_tools_endpoint(self):
        """Test tools endpoint has all 11 tools"""
        response = self.session.get(f"{self.service_url}/tools", timeout=10)
        self.assertEqual(response.status_code, 200, "/tools should return 200")
        
        data = response.json()
        tools = data.get("tools", [])
        self.assertEqual(len(tools), 11, "Should have exactly 11 tools")
        
        # Check tool names
        expected_tools = [
            "list_boards", "get_board", "create_board", "update_board",
            "get_lists", "create_list",
            "get_cards", "create_card", "update_card",
            "add_member_to_card", "search_cards"
        ]
        tool_names = [t.get("name") for t in tools]
        for tool_name in expected_tools:
            self.assertIn(tool_name, tool_names, f"Should have {tool_name} tool")
    
    def test_05_tools_optional_credentials(self):
        """Test that credentials are optional for all tools"""
        response = self.session.get(f"{self.service_url}/tools", timeout=10)
        data = response.json()
        tools = data.get("tools", [])
        
        for tool in tools:
            name = tool.get("name")
            schema = tool.get("inputSchema", {})
            required = schema.get("required", [])
            
            # api_key and token should NOT be in required list
            has_api_key = "api_key" in required
            has_token = "token" in required
            
            self.assertFalse(has_api_key, f"{name} should not require api_key")
            self.assertFalse(has_token, f"{name} should not require token")
    
    def test_06_mcp_initialize(self):
        """Test MCP initialize request"""
        mcp_request = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "initialize",
            "params": {
                "protocolVersion": "2024-11-05",
                "clientInfo": {"name": "test", "version": "1.0"}
            }
        }
        
        response = self.session.post(
            f"{self.service_url}/mcp",
            json=mcp_request,
            timeout=10,
            headers={"Content-Type": "application/json"}
        )
        self.assertEqual(response.status_code, 200, "MCP endpoint should return 200")
        
        data = response.json()
        self.assertEqual(data.get("jsonrpc"), "2.0", "Should be JSON-RPC 2.0")
        self.assertIn("result", data, "Should have result")
        
        result = data.get("result", {})
        self.assertIn("capabilities", result, "Result should have capabilities")
    
    def test_07_mcp_tools_list(self):
        """Test MCP tools/list request"""
        mcp_request = {
            "jsonrpc": "2.0",
            "id": 2,
            "method": "tools/list"
        }
        
        response = self.session.post(
            f"{self.service_url}/mcp",
            json=mcp_request,
            timeout=10,
            headers={"Content-Type": "application/json"}
        )
        self.assertEqual(response.status_code, 200, "MCP endpoint should return 200")
        
        data = response.json()
        result = data.get("result", {})
        tools = result.get("tools", [])
        
        self.assertEqual(len(tools), 11, "MCP should report 11 tools")


if __name__ == "__main__":
    unittest.main()
'''

# Write the test file
with open(test_file_path, "w") as f:
    f.write(test_code)

print(f"✅ Test file saved to: {test_file_path}")
print(f"📁 File size: {test_file_path.stat().st_size} bytes")
print("\n📋 To run the tests, use:")
print(f"   python -m unittest tests.test_cloud_run_integration -v")

✅ Test file saved to: /Users/shlomisha/Documents/vscodeprojects/Trello/tests/test_cloud_run_integration.py
📁 File size: 5947 bytes

📋 To run the tests, use:
   python -m unittest tests.test_cloud_run_integration -v


## Section 8: Run and Validate All Tests

In [27]:
# Run all unit tests
print("\n" + "="*80)
print("🧪 RUNNING UNIT TESTS")
print("="*80 + "\n")

# Create test suite
loader = unittest.TestLoader()
suite = loader.loadTestsFromTestCase(TestTrelloMCPCloudRun)

# Run tests with verbose output
runner = unittest.TextTestRunner(verbosity=2)
result = runner.run(suite)

# Summary
print("\n" + "="*80)
print("📊 TEST SUMMARY")
print("="*80)
print(f"✅ Tests run: {result.testsRun}")
print(f"✅ Failures: {len(result.failures)}")
print(f"✅ Errors: {len(result.errors)}")

if result.wasSuccessful():
    print("\n🎉 ALL TESTS PASSED!")
    print("\n✅ Trello MCP Cloud Run service is fully operational")
    print("✅ Interactive login feature is working")
    print("✅ All 11 Trello tools are available with optional credentials")
    print("✅ MCP protocol is correctly implemented")
else:
    print("\n⚠️  Some tests failed. Check the output above for details.")

test_01_health_endpoint (__main__.TestTrelloMCPCloudRun.test_01_health_endpoint)
Test health endpoint returns 200 and correct structure ... 


🧪 RUNNING UNIT TESTS



ok
test_02_root_endpoint (__main__.TestTrelloMCPCloudRun.test_02_root_endpoint)
Test root endpoint and feature flags ... test_02_root_endpoint (__main__.TestTrelloMCPCloudRun.test_02_root_endpoint)
Test root endpoint and feature flags ... ok
test_03_auth_login_endpoint (__main__.TestTrelloMCPCloudRun.test_03_auth_login_endpoint)
Test interactive login endpoint ... ok
test_03_auth_login_endpoint (__main__.TestTrelloMCPCloudRun.test_03_auth_login_endpoint)
Test interactive login endpoint ... ok
test_04_tools_endpoint (__main__.TestTrelloMCPCloudRun.test_04_tools_endpoint)
Test tools endpoint has all 11 tools ... ok
test_04_tools_endpoint (__main__.TestTrelloMCPCloudRun.test_04_tools_endpoint)
Test tools endpoint has all 11 tools ... ok
test_05_tools_optional_credentials (__main__.TestTrelloMCPCloudRun.test_05_tools_optional_credentials)
Test that credentials are optional for all tools ... ok
test_05_tools_optional_credentials (__main__.TestTrelloMCPCloudRun.test_05_tools_optional_credent


📊 TEST SUMMARY
✅ Tests run: 7
✅ Failures: 0
✅ Errors: 0

🎉 ALL TESTS PASSED!

✅ Trello MCP Cloud Run service is fully operational
✅ Interactive login feature is working
✅ All 11 Trello tools are available with optional credentials
✅ MCP protocol is correctly implemented
